# 1. Демодуляция сигнала

In [4]:
import scipy.io.wavfile as wav
import scipy.signal as signal
import scipy.fft
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math
from PIL import Image
from copy import copy

In [5]:
def get_ampl(t1, t2, a1, a2, w):
    alpha = 0.0
    a = w*(t1 - t2)/2
    b = w*(t1 + t2)/2
    if(a1 == 0):
        alpha = -w*t1
        return (a2/math.sin(w*t2 + alpha), alpha)
    if(a2 == 0):
        alpha = -w*t2
        return (a1/sin(w*t1 + alpha), alpha)
    if((a1 + a2) == 0):
        alpha = -a
    else:
        if(a1 == a2):
            alpha = -math.atan(((a1-a2)/(a1+a2))/math.tan(a)) - b - math.pi/2
        else:
            alpha = math.atan(((a1+a2)/(a1-a2))/(-math.tan(a+ math.pi/2))) - b
    if(abs(a1)>abs(a2)):
        A = a1/math.sin(w*t1 + alpha)
    else:
        A = a2/math.sin(w*t2 + alpha)
    
    return (A, alpha)

fs, data = wav.read('signal.wav')

data = [int(el) for el in data]
avg = np.mean(data)
data = [abs(el - avg) for el in data]

f=4160.0
fc = 2400.0

resample = []
i = 0 
current_interval = 0
stat = []
buf = []

while True:
    if(i >= len(data)):
        resample.append(max(buf))
        break
    if((((i*f/fs)-current_interval) // 1) == 0.0):
        buf.append(data[i]) 
    else:
        buf.sort()
        a1 = buf[-2]
        a2 = buf[-1]
        t1 = (i-len(buf))/fs
        t2 = (i-len(buf)+1)/fs
        A = get_ampl(t1, t2, a1, a2, fc*2*math.pi)
        stat.append(A)    
        resample.append(abs(A[0]))
        buf.clear()
        buf.append(data[i])
        current_interval += 1
    i+=1
  
shift = 1.0/f # f = t / N
t1 = [i/fs for i in range(20)]
t2 = [i/f + shift/2.0 for i in range(10)]
plt.plot(t1, data[0:20])
plt.plot(t2, resample[0:10])
for el in t2:
    plt.plot([el+shift/2.0, el+shift/2.0], [-50, 50], '-r')
plt.xlabel("Samples")
plt.ylabel("Amplitude")
plt.title("Signal")
plt.show()

data_am1 = resample 

mx = max(data_am1)
mn = min(data_am1)

data_am1 = [math.ceil(255*(el-mn)/(mx-mn)) for el in data_am1]
frame_width = int(0.5*f)

w, h = frame_width, len(data_am1)//frame_width
image = Image.new('RGB', (w, h))

px, py = 0, 0
for p in range(len(data_am1)):
    lum = data_am1[p]
    if lum < 0: lum = 0
    if lum > 255: lum = 255
    image.putpixel((px, py), (0, lum, 0))
    px += 1
    if px >= w:
        if (py % 50) == 0:
            print(f"Line saved {py} of {h}")
        px = 0
        py += 1
        if py >= h:
            break

%matplotlib qt
plt.imshow(image)
plt.show()

<ipython-input-5-39b395710303>:25: WavFileWarning: Reached EOF prematurely; finished at 14860332 bytes, expected 14860337 bytes from header.
  fs, data = wav.read('signal.wav')


Line saved 0 of 1347
Line saved 50 of 1347
Line saved 100 of 1347
Line saved 150 of 1347
Line saved 200 of 1347
Line saved 250 of 1347
Line saved 300 of 1347
Line saved 350 of 1347
Line saved 400 of 1347
Line saved 450 of 1347
Line saved 500 of 1347
Line saved 550 of 1347
Line saved 600 of 1347
Line saved 650 of 1347
Line saved 700 of 1347
Line saved 750 of 1347
Line saved 800 of 1347
Line saved 850 of 1347
Line saved 900 of 1347
Line saved 950 of 1347
Line saved 1000 of 1347
Line saved 1050 of 1347
Line saved 1100 of 1347
Line saved 1150 of 1347
Line saved 1200 of 1347
Line saved 1250 of 1347
Line saved 1300 of 1347


## 2. Нормализация данных

In [6]:
mx = max(resample)
mn = min(resample)
resample = [round(255*(el-mn)/(mx-mn)) for el in resample]

# Сделаем картинку ровной

## 1.1 Реализуем функции для поиска синхроимпульса

In [7]:
def scan(data, start, pattern):
    sm = 0.0
    for i in range(len(pattern)):
        sm += (data[start+i] - pattern[i])**2
    return math.sqrt(sm)

def get_min_pos(arr):
    mn = arr[0]
    pos = 0
    for i in range(len(arr)):
        if(arr[i] < mn):
            mn = arr[i]
            pos = i
    return pos

def get_sync_pulse(data, pattern):
    stat1 = []
    for i in range(2080):
        stat1.append(scan(data, i, pattern1))
    return get_min_pos(stat1)


## 1.2 Выровняем изображение

In [8]:
resample1 = resample
s = '000011001100110011001100110011000000000'
s2 = '000011100111001110011100111001110011100'

pattern1 = []
pattern2 = []

for el in s:
    pattern1.append(255*int(el)) 
for el in s2:
    pattern2.append(255*int(el))
img = []
start = get_sync_pulse(resample1, pattern1)
img = img + resample1[start:start+2080]
start = start + 2080
while True:
    if(start+6000 >= len(resample1)):
        break
    start = start +  get_sync_pulse(resample1[start+2080:], pattern1)
    img = img + resample1[start:start+2080]
    start = start + 2080
    print("{:.2f}".format(float(start)/float(len(resample1))), end='\r')


In [9]:
def get_image(img, width):
    w, h = width, len(img)//width
    image = Image.new('RGB', (w, h))
    
    px, py = 0, 0
    for p in range(len(img)):
        lum = img[p]
        if lum < 0: lum = 0
        if lum > 255: lum = 255
        image.putpixel((px, py), (0, int(lum), 0))
        px += 1
        if px >= w:
            if (py % 50) == 0:
                print(f"Line saved {py} of {h}")
            px = 0
            py += 1
            if py >= h:
                break
    return image

image = get_image(img, 2080)
%matplotlib qt

plt.imshow(image)
plt.show()

Line saved 0 of 1088
Line saved 50 of 1088
Line saved 100 of 1088
Line saved 150 of 1088
Line saved 200 of 1088
Line saved 250 of 1088
Line saved 300 of 1088
Line saved 350 of 1088
Line saved 400 of 1088
Line saved 450 of 1088
Line saved 500 of 1088
Line saved 550 of 1088
Line saved 600 of 1088
Line saved 650 of 1088
Line saved 700 of 1088
Line saved 750 of 1088
Line saved 800 of 1088
Line saved 850 of 1088
Line saved 900 of 1088
Line saved 950 of 1088
Line saved 1000 of 1088
Line saved 1050 of 1088


## 2.1 Выберем часть изображения так, чтобы телеметрия начиналась с начала новой полосы(выбираем диапазон, опираясь на картинку из предыдущего блока)

In [10]:
img1 = img[273*2080:]
image = get_image(img1, 2080)
%matplotlib qt

plt.imshow(image)
plt.show()

Line saved 0 of 815
Line saved 50 of 815
Line saved 100 of 815
Line saved 150 of 815
Line saved 200 of 815
Line saved 250 of 815
Line saved 300 of 815
Line saved 350 of 815
Line saved 400 of 815
Line saved 450 of 815
Line saved 500 of 815
Line saved 550 of 815
Line saved 600 of 815
Line saved 650 of 815
Line saved 700 of 815
Line saved 750 of 815
Line saved 800 of 815


## 2.2 Выделим телеметрические данные из изображения

In [11]:
telemetry = [[], []]
i = 0
while((i + 1040+995+45)<len(img1)):
    telemetry[0] = telemetry[0] + img1[i + 995:i+995+45]
    telemetry[1] = telemetry[1] + img1[i + 1040 + 995:i + 1040 + 995 + 45]
    i += 2080

In [12]:
image=get_image(telemetry[0], 45)
%matplotlib qt
wd = len(telemetry)//45
plt.imshow(image)
plt.show()

Line saved 0 of 814
Line saved 50 of 814
Line saved 100 of 814
Line saved 150 of 814
Line saved 200 of 814
Line saved 250 of 814
Line saved 300 of 814
Line saved 350 of 814
Line saved 400 of 814
Line saved 450 of 814
Line saved 500 of 814
Line saved 550 of 814
Line saved 600 of 814
Line saved 650 of 814
Line saved 700 of 814
Line saved 750 of 814
Line saved 800 of 814


## 2.3 Для каждого блока телеметрии найдем медиану и установим ее в качестве основного значения

In [13]:
values = [[], []]
for j in range(len(telemetry)):
    for i in range(0, len(telemetry[j]), 45*8):
        values[j].append(int(math.ceil(statistics.median(telemetry[j][i:i+45*8]))))

In [14]:
test = [[], []]
for i in range(len(telemetry)):
    test[i] = copy(telemetry[i])
for j in range(len(telemetry)):  
    for i in range(len(test[j])-1):
        ind = (i+1)//(45*8)
        test[j][i] = values[j][ind]
    

In [15]:
image=get_image(test[1], 45)
%matplotlib qt
wd = len(test)//45
plt.imshow(image)
plt.show()

Line saved 0 of 814
Line saved 50 of 814
Line saved 100 of 814
Line saved 150 of 814
Line saved 200 of 814
Line saved 250 of 814
Line saved 300 of 814
Line saved 350 of 814
Line saved 400 of 814
Line saved 450 of 814
Line saved 500 of 814
Line saved 550 of 814
Line saved 600 of 814
Line saved 650 of 814
Line saved 700 of 814
Line saved 750 of 814
Line saved 800 of 814


# Сохраним данные

In [16]:
np.save("image", np.array(img1))
np.save("w1", np.array(test[0]))
np.save("w2", np.array(test[1]))